## Ensemble Autogluon + Promedio 

En este script se realiza un ensemble entre las predicciones de autogluon en las que logramos menor error y el promedio de 12 meses. La estrategia aplicada es la siguiente en base a las predicciones de febrero 2019 , se identifico un grupo de product id en los cuales el error era menor si se utilizaba como prediccion el promedio. 

In [1]:
import pandas as pd
import numpy as np

In [3]:
# 1. Cargar los archivos

df_autogluon_23 = pd.read_csv("predicciones_febrero2020_01-07-25_2.csv")
df_promedios = pd.read_csv("promedio_12m_febrero2020_2.csv")

product_mejor_autogluon = pd.read_csv("product_id_autogluon_mejor.csv")
product_preciso = pd.read_csv("casos_mejor_promedio_total.csv")

In [5]:
# Crear sets para facilitar la selección
ids_mejor_autogluon = set(product_mejor_autogluon["product_id"])
ids_mejor_preciso = set(product_preciso["product_id"])

# Inicializar el dataframe ensamblado con los product_id únicos
df_ensamblado = pd.DataFrame({'product_id': df_autogluon_23['product_id']})

# Asignar las tn según la lógica
def obtener_tn(row):
    pid = row['product_id']
    if pid in ids_mejor_autogluon:
        return df_autogluon_23.loc[df_autogluon_23['product_id'] == pid, 'tn'].values[0]
    elif pid in ids_mejor_preciso:
        return df_promedios.loc[df_promedios['product_id'] == pid, 'tn'].values[0]
    else:
        tn_auto = df_autogluon_23.loc[df_autogluon_23['product_id'] == pid, 'tn'].values[0]
        tn_prom = df_promedios.loc[df_promedios['product_id'] == pid, 'tn'].values[0]
        return np.mean([tn_auto, tn_prom])

df_ensamblado['tn'] = df_ensamblado.apply(obtener_tn, axis=1)

# Mostrar el resultado
print(df_ensamblado.head())
print("Cantidad de productos en el ensamblado:", len(df_ensamblado))

   product_id           tn
0       20001  1289.854612
1       20002  1043.714777
2       20003   730.515933
3       20004   542.798590
4       20005   534.809802
Cantidad de productos en el ensamblado: 780


In [6]:
df_ensamblado.to_csv("df_ensamblado_final.csv", index=False)
print("Archivo 'df_ensamblado_final.csv' guardado exitosamente.")   

Archivo 'df_ensamblado_final.csv' guardado exitosamente.
